In [ ]:
from langchain_tavily import TavilySearch
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

web_search = TavilySearch(max_results=2)
tools = [web_search]
# web_search.invoke("What's a 'node' in LangGraph?")

In [74]:

from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.0-flash")

In [75]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
# 对于在第一个教程中创建的 StateGraph ，在 LLM 上添加 bind_tools
# 这能让 LLM 知道如果它想要使用tools中的工具，应该使用正确的 JSON 格式
# 决定是否调用工具由llm决定
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    # 把整个state的message都传过去，相当于为llm增加记忆
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [ ]:
# 现在，创建一个函数来运行被调用的工具。通过将工具添加到一个名为 BasicToolNode 
# 的新节点来实现，该节点检查状态中的最新消息，如果消息包含 tool_calls ，则调用
# 工具。它依赖于 LLM 的 tool_calling 支持
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict): # 这里写 dict 只是类型提示
        # 实际上，无论传入什么类型，__call__ 都会执行
        # 但是下面这行代码要求 inputs 必须有 .get() 方法（即字典）
        # 使用get方法，遇到不存在的key会返回None或者默认值，使用[]遇到不存在key会报key error
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

# 这里创建实例调用init
tool_node = BasicToolNode(tools=[web_search])
# result = tool_node(some_input)  # ← 这里才是调用 __call__
graph_builder.add_node("tools", tool_node)

In [69]:
'''
定义 conditional_edges
边将控制流从一个节点路由到下一个节点。条件边从一个节点开始，通常包含"if"语句，根据当前图状态路由到不同的节点。
这些函数接收当前图 state 并返回一个字符串或字符串列表，指示下一个要调用的节点。

接下来，定义一个名为 route_tools 的路由函数，该函数用于检查聊天机器人的输出中是否包含 tool_calls。
通过调用 add_conditional_edges 将此函数提供给图，这会告诉图每当 chatbot 节点完成时，需要检查此函数以确定下一步的去向。

条件存在工具调用时将路由到 tools ，不存在时将路由到 END 。因为条件可以返回 END ，所以这次不需要显式设置 finish_point
'''
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "edge_tools"
    return "edge_end"


# The `tools_condition` function returns "edge_tools" if the chatbot asks to use a tool, and "edge_end" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",  # 源节点名称
    route_tools,    # 决定路径的函数
    {"edge_tools": "tools", "edge_end": END}, # 路径映射字典，根据route_tools返回的结果确定下一个Node
)
# Any time a tool is called, we return to the chatbot to decide the next step
# 将 tools 和 chatbot 这两个bot相连
graph_builder.add_edge("tools", "chatbot")
# 将 chatbot 与 开始节点相连
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [70]:
from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [71]:
'''
Now you can ask the chatbot questions outside its training data
'''
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: 
Assistant: {"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectures to model and manage the intricate relationships between various components of an AI agent workflow. LangGraph illuminates the processes within an AI workflow, allowing full transparency of the agent\u2019s state. By combining these technologies with a set of APIs and tools, LangGraph provides users with a versatile platform for developing AI solutions and workflows including chatbots, state graphs and other agent-based systems. Nodes: In LangGraph, nodes represent individual components or agents within an AI workflow. LangGraph uses enhanced decision-making by 

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from dotenv import load_dotenv
load_dotenv()

# 构建state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 初始化工具
tool = TavilySearch(max_results=2)
tools = [tool]

# 初始化模型 绑定工具
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

# 定义Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
# 增加Node
graph_builder.add_node("chatbot", chatbot)

# 定义工具Node
tool_node = ToolNode(tools=[tool])
# 增加工具Node
graph_builder.add_node("my_tools", tool_node)

# 增加条件边
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,    # langgraph自带的工具条件函数返回字符串是写死的，因此下面的映射字典，只能写tools和__end__
    {"tools": "my_tools", "__end__": END} # 如果不需要工具调用 直接结束
)
# 增加普通边
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("my_tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()